### PBL02


- **제출자: AI26_오준서**  
- **제출일: 2025-06-12**  
- **PBL02 : 로그 파일을 활용한 IP 접속 분석 ( 정규 표현식과 데이터 구조를 이용한 접속 IP 분석)**
    - **1.사용자 입력: 로그 파일 경로**
    - **2.IP 주소 추출 및 빈도 계산**
    - **3.상위 3개 IP 주소 출력**
    - **4.분석 결과 CSV 저장 (IP_analysis.csv)**
    
- **해결 가이드**
    - **re 모듈을 사용해 IP 정규 표현식 정의**
    - **Counter로 빈도 계산**
    - **csv 모듈로 결과 저장 (utf-8-sig)**
    - **os.path.exists() 로 파일 존재 여부 확인 -> csv로 저장하고 확인.** 

#### re 모듈에 대해 간단한 정리
- https://docs.python.org/3/library/re.html
- regular expression 을 의미한다. (정규 표현식)
- 표준 라이브러리 (import re  해서 사용하면 된다)

- 정규 표현식 구문
    - \ : 특수 형식을 나타내거나 특수 문자를 특별한 의미 없이 사용할 수 있도록 한다.  (. 은 임의의 한 문자를 의미하기에 \. 이렇게 해야지 .을 구별할 수 있다)
    - ? : 결과 RE가 이전 RE의 0또는 1번 반복과 일치하도록 한다
    - {m} : 이전 RE의 정확히 m개 사본이 일치하도록 지정
    - {m,n} : m이상, n이하 반복
    - (...) : 괄호 안에 있는 정규 표현식과 일치, 그룹의 시작과 끝을 의미 
    - \b : 단어의 시작이나 끝에 있는 문자열만 일치 (경계를 의미)
    - \d : 숫자를 의미. [0-9] 와 동일.
    - | : A 또는 B와 일치하는 정규표현식을 생성
    - (?:...) : 비캡쳐 버전. 그룹이 매칭한 부분 문자열은 이후에 추출하거나 참조할 수 없다. (부분이 아니라 문자열을 뽑고 싶을 때 사용!)

- IPv4 주소를 추출하기 위한 정규표현식
- **IPv4 주소는 32bit 체계, 4개의 옥텟으로 사용**
    - r'(?:\d{1,3}\.){3}\d{1,3}'     
        - X.X.X.X    -> 간단하게 구현. 범위가 0-999 까지여서 형태와 맞지x

    - (25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)(\.(25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)){3}
        - **0-255 까지 제한. 리딩 제로 방지 (01, 001)**
        - **[0-9] 와 \d 는 같은 의미 이기에 간결하게 \d 로 표현**
        - **여러개의 정규표현식을 | 기호로 구분해서 나열 (255-0)**
        - **양쪽 끝에 \b 또는 ^, $ 를 추가할 수 있다.**

- 문자열 검색을 위한 4가지 메서드
    - match() : 문자열의 처음부터 정규식과 매치되는지 조사
    - search() : 문자열 전체를 검색하여 정규식과 매치되는지 조사
    - findall() : 정규식과 매치되는 모든 문자열을 리스트로 리턴
    - finditer() : 정규식과 매치되는 모든 문자열을 반복 가능한 객체로 리턴

    - **간단한 사용 법**
        - match = re.search(pat, str)
        -   만약 검색에 성공할면 match 객체 반환, 아니면 None 반환

- 정규식 컴파일을 위한 옵션
    - DOTALL (.) - dot 이 줄바꿈 문자를 포함해 모든 문자와 매치
    - IGNORECASE (I) - 대소문자에 관계없이 매치
    - MULTILINE (M) - 여러 줄과 매치. ^, $ 메타 문자 사용과 관계
    - VERBOSE (X) - verbose 모드를 사용. 정규식을 보기 편하게 만든다. 주석 등을 사용 할 수 있게 된다.

- 분석은 빈도 수에 따른 상위 3개 추출
- 각 IP 의 로그를 csv 파일로 저장 
    - 해당 로그들을 가독성 있게 정리( 전체 로그를 저장 + 빈도) 

In [ ]:
# 실제 구현을 위한 간단한 코드.

from collections import Counter
import csv
import re
import os

log_path = '20250613_091920_sample_log_file.log'

# IPv4 정규 표현식 정의
# 0.0.0.0 ~ 255.255.255.255
ipv4 = r'\b(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)(?:\.(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)){3}\b'

if not os.path.exists(log_path):
    print("로그 파일이 존재하지 x")

with open(log_path, 'r', encoding='utf-8') as f:
    log_total = f.read() # 파일의 전체 내용을 str 으로 읽어서 저장.

# print(log_total)
# 전체 로그에서 IP 추출
# 리스트로 반환환
ip_list = re.findall(ipv4, log_total)
# print(ip_freq)
print(type(ip_list))

# Counter 를 이용해서 빈도 확인
ip_freq =[]
ip_freq = Counter(ip_list)
print(ip_freq)
print(type(ip_freq))
ip_top3 = ip_freq.most_common(3) # 상위 3개 RE, list 형태
print(ip_top3, type(ip_top3))

In [26]:
# 실제 구현ㄴ
from collections import Counter, defaultdict
import csv
import re
import os

class LogExtractAnalysis:
    def __init__(self, log_path):
        # 로그 파일 경로
        self.log_path = log_path
        
        # Ipv4 정규표현식 패턴
        self.ipv4 = r'\b(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)(?:\.(?:25[0-5]|2[0-4]\d|1\d\d|[1-9]\d?|0)){3}\b'
        
        # 추출된 IP 주소 리스트 (Counter 사용을 위해)
        self.ip_list = []
        
        # 전체 IP 빈도수 (Counter의 RE가 저장되는 리스트)
        self.ip_freq = []
        
        # 빈도 상위 3개의 튜플 리스트 ('ip주소' : 빈도)
        self.ip_top3 = []
        
        # self.ip_log_total = {} # 이렇게 하니까 KeyError 발생
        self.ip_log_total = defaultdict(list)
    
    # 로그 파일을 한줄 씩 읽어서 리스트로 반환하는 함수    
    def log_read(self):
        # 로그 파일이 존재하는지 확인
        if not os.path.exists(self.log_path):
            raise FileNotFoundError(f"{self.log_path} 가 존재하지 않습니다.")
        with open(self.log_path, 'r', encoding='utf-8') as f:
            # log_total = f.read()
            # return log_total     # 로그 파일의 전체 내용을 str 형태로 RE 
            return f.readlines() # 줄 단위로 리스트 반환
    
    # 로그에서 ip 추출 및 관련 나머지 로그 저장하는 함수
    def log_extract_ip(self):
        # 단순히 csv로 저장할때 ip와 빈도 뿐만 아니라 해당하는 전체 로그도 저장하기 위해
        # search() 를 반복적으로 사용하면서 저장
        log_lines = self.log_read()
        for line in log_lines:
            match = re.search(self.ipv4, line)
            if match:
                ip = match.group() # 정규식으로 추출된 IP
                self.ip_list.append(ip) # 추출된 IP 저장
                self.ip_log_total[ip].append(line.strip()) # 관련 IP 로그 저장
            
    def log_top3_extract_view(self):
        # ip_top3에 빈도수가 가장 많은 3개의 ip주소 저장(리스트 형태)
        self.ip_freq = Counter(self.ip_list) # 빈도수 저장
        self.ip_top3 = self.ip_freq.most_common(3) # 상위 3개 추출
        for ip, freq in self.ip_top3:
            print(f"{ip} : {freq} 회") # 출력
       
    def save_to_csv(self, output_csv = 'ip_analysis.csv'):
        # 상위 3개 IP에 대한 결과 CSV로 저장
        with open(output_csv, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['IP address','freqence', 'total_log']) # 헤더
            
            for ip, freq in self.ip_top3:
                logs_total = "\n".join(self.ip_log_total[ip]) # 로그 합치고
                writer.writerow([ip, freq, logs_total]) # 한 행씩 저장
        print(f"\n분석 결과가 csv 파일로 저장되었습니다. -> {output_csv}")
        
        
if __name__ == '__main__':
    log_path = input("로그 파일 경로 입력: ").strip() # 사용자로부터 파일 경로 입력 받기
    log = LogExtractAnalysis(log_path) # 객체 생성
    log.log_extract_ip() # IP 로그 추출
    log.log_top3_extract_view() # 상위 3개 출력 
    log.save_to_csv() # 결과 출력 및 CSV 저장


203.0.113.5 : 17 회
192.168.0.3 : 14 회
192.168.0.2 : 14 회

분석 결과가 csv 파일로 저장되었습니다. -> ip_analysis.csv
